In [ ]:
import config
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import datetime
from utils.data_exploration_utils import drop_unnamedcolumn,  investigate_data, plot_hist, scatterplot, missing_from_df

In [ ]:
today = datetime.date.today()

base_dir = config.RAW_DATA_PATH
proc_dir = config.PROC_DATA_PATH

folder = None

if folder is not None:
    save_dir = os.path.join(proc_dir, folder)
    save_dir2 = save_dir
else:
    save_dir = os.path.join(proc_dir, f"{today}_data_exploration")
    save_dir2 = os.path.join(proc_dir, '2025-07-14_data_exploration')

os.makedirs(save_dir, exist_ok=True)

first_leg = "inmodi_data_first_visit.csv"
second_leg = "inmodi_data_second_visit.csv"

df_first = pd.read_csv(os.path.join(base_dir, first_leg), sep=",")
df_second = pd.read_csv(os.path.join(base_dir, second_leg), sep=",")

#TODO: import newest KL scores
kl = pd.read_csv(os.path.join(base_dir, 'brul_knee_annotations.csv'))

# Data Cleaning

In [ ]:
# remove unnamed columns
try:
    df_first = drop_unnamedcolumn(df_first)
    df_second = drop_unnamedcolumn(df_second)
except Exception as e:
    print(f"Error dropping unnamed columns: {e}")

In [ ]:
print("Columns in first visit data:")
print(df_first.columns.tolist())

print("\nColumns in second visit data:")
print(df_second.columns.tolist())

COMI_score missing from second leg

## Missing Values & Duplicates

In [ ]:
df_first_nanids = investigate_data(df_first) #save_path=save_dir, save_name="inmodi_data_first_visit"

In [ ]:
# missing values in KOOS data?
# for i in df_first_nanids:
#     display(df_first[df_first["record_id"] == i])

In [ ]:
df_second_nanids = investigate_data(df_second) #save_path=save_dir, save_name="inmodi_data_second_visit"

In [ ]:
# for i in df_second_nanids:
#     display(df_second[df_second["record_id"] == i])

For first leg, ce_fm, ce_pain_r, ce_pain_l, COMI_score, UCLA_score, KOOS scores and rx_ap_kl_left appear to possibly have missing values.

Also we have a unique record_id count of 121, but 122 record_ids (rows), therefore we might have duplicates.

## Quick Fix: Removing duplicates

In [ ]:
print("Number of rows: ", len(df_first))
df_first = df_first.drop_duplicates(subset='record_id', keep='first')
print("Number of rows after dropping duplicates: ", len(df_first))

## Quick Fix: Fix Unique case

In [ ]:
display(df_second[df_second['ce_fm']=='35.4, 26.4'])
outl_index = df_second[df_second['ce_fm']=='35.4, 26.4'].index

print(f"Location: {outl_index}")

In [ ]:
df_second.loc[outl_index, 'ce_fm'] = 35.4

In [ ]:
display(df_second.iloc[outl_index])

## KL-score

In [ ]:
kl.describe(include='all')

In [ ]:
kl_nanids = investigate_data(kl, id_col='name')

# Patient-Related Information

This includes patient information, such as age, bmi, body fat, pain and sex. Additionally, we will look at the aggregated questionnaire scores.

*Height and Weight was purposefully omitted in this part, since it is already covered in the BMI, which takes the height into account when evaluating the weight.*

**Columns Patient-Related Data**
* `record_id`: id column
* `age`: patient age (at time of study?)
* `ce_height`
* `ce_weight`
* `ce_bmi`
* `ce_fm`: body fat percentage (%)
* `ce_pain_r`: patient-reported pain (right leg)
* `ce_pain_l`: patient-reported pain (left leg)
* `gender`
* `COMI_score`: Core Outcome Measures Index
* `OKS_score`: Oxford Knee Score (12 Questions)
* `UCLA_score`: UCLA-Activity Index (1 Question)
* `FJS_score`: Forgotten Joint Score (12 Questions)
* `KOOS_pain`: Knee Injury and OA Outcome Score - Pain Section (9 Questions)
* `KOOS_symptoms`: Symptoms Section (7 Questions)
* `KOOS_sport`: Sport Section (5 Questions)
* `KOOS_adl`: Daily Activities Section (17 Questions)
* `KOOS_qol`: Quality of Living Section (4 Questions)
* `rx_ap_kl_left`: KL-Score (left leg)
* `rx_ap_kl_right`: KL-Score (right leg)

The various scores are aggregated in this dataset already.

In [ ]:
df_first['visit'] = 1
df_second['visit'] = 2

pi = pd.concat([df_first, df_second], ignore_index=True)
print("Number of rows after concatenation: ", len(pi))

pi = pi.astype({'ce_fm':'float64'})
pi= pi.drop(columns=['rx_ap_kl_left', 'rx_ap_kl_right'])

# Columns that we will analyze
l_columns = ['age', 'ce_bmi', 'ce_fm',
       'gender',  'OKS_score',
       'UCLA_score', 'FJS_score', 'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport',
       'KOOS_adl', 'KOOS_qol'] #'COMI_score',

In [ ]:
def plot_hist(df, column, colname, title = None, xlabel = None, y_label = "Frequency", stat = 'frequency', figsize=(10, 6), hue= None, multiple='dodge', bins = 30, kde=False):
    sns.set_theme(style="whitegrid", font_scale=1.2)
    plt.figure(figsize=figsize)
    if hue is None:
        sns.histplot(df[column], bins=bins, stat=stat, kde=kde)
    else:
        sns.histplot(data=df, x = column, bins=bins, stat=stat, hue = hue, multiple=multiple, kde=kde)
    plt.title(title if title else f"Distribution of {colname}")
    plt.xlabel(xlabel if xlabel else colname)
    plt.ylabel(y_label)
    plt.show()

In [ ]:
lcols = ['age',
    'ce_bmi', 'ce_fm', 'gender']

namecol = [ 'Age',
    'BMI', 'Body Fat Percentage', 'Sex']
for i, col in enumerate(lcols):
    if col != 'gender':
        plot_hist(pi, col, colname=namecol[i], figsize=(10, 6), stat='density', y_label = 'Density', bins=10, kde=True)
    elif col == 'gender':
        plot_hist(pi, col, colname=namecol[i], figsize=(10, 6), stat='density', y_label = 'Density', bins=2, kde=False)

In [ ]:
pi['gender'].value_counts()

In [ ]:
l_columns2 = ['age', 'ce_bmi', 'ce_fm',
         'OKS_score',
       'UCLA_score', 'FJS_score', 'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport',
       'KOOS_adl', 'KOOS_qol'] #'COMI_score',
for col in l_columns2:
    melted_df = pi.melt(id_vars='gender', value_vars=col, var_name='Variable', value_name='Count')
    plt.figure(figsize=(12, 6))
    sns.violinplot(data=melted_df, x='Variable', y='Count', hue='gender', split=True, inner='quartile')
    plt.title(f"Distribution of {col} by Gender")
    plt.xticks(rotation=45)
    plt.show()

Pain for left and right leg needs to be concated, since we are interested in overall pain distribution, not specifically pain distribution of left and right leg.

### Patient-reported Pain

In [ ]:
pain_df = pi.melt(id_vars=['record_id', 'visit'], value_vars=['ce_pain_l', 'ce_pain_r'], var_name='side', value_name='pain')

print()
print(f"Length of pivoted df: {len(pi)}")
print(f"Length of unpivoted df: {len(pain_df)}")

pain_df['side'] = pain_df['side'].str.replace('ce_pain_', '')

display(pain_df.head())


In [ ]:
pi2 = pi.drop(columns=['ce_pain_l', 'ce_pain_r'])

pi2 = pi2.drop_duplicates(subset=['record_id', 'visit'], keep='first')

print(f"PI2 DF shape: {pi2.shape}")

In [ ]:
print(f"PI DF shape: {pi.shape}")
pi2 = pain_df.merge(pi2, on=['record_id', 'visit'], how='left')

print()
print(f"PI DF shape after merging pain data: {pi2.shape}")

In [ ]:
display(pi2.head())

In [ ]:
pi2.drop(columns='COMI_score', inplace=True)

In [ ]:
count_recordid = pi2['record_id'].value_counts().reset_index()

print(f"These are the record ids with too many rows:")
display(count_recordid[count_recordid['count'] > 4])

In [ ]:
pain_df['pain'].max()

*9 bins, because highest score is 9 and it is an int value.*

In [ ]:
# plot_hist(pain_df, column='pain', y_label='Percentage', stat='percent', 
#           kde=True, bins=9)

In [ ]:
# plot_hist(pi2, column='pain', y_label='Percentage', stat='percent', hue = 'gender',
#           multiple='dodge', kde=True, bins=9)

In [ ]:
pi2.head()

In [ ]:
# #TODO: save cleaned pi version
# pi.to_csv(os.path.join(save_dir, "inmodi_data_personalinformation.csv"), index=False)

# #TODO: save cleaned unpivoted pi version
# pi2.to_csv(os.path.join(save_dir, "inmodi_data_personalinformation_unpivoted.csv"), index=False)

# KL Score

## Overall distribution

In [ ]:
display(kl['KL-Score'].value_counts().reset_index())

print(f"Number of samples: {len(kl)}")

## Inconsistent KL-Score by Patient Identification

In [ ]:
kl[['record_id', 'visit', 'side']] = kl['name'].str.split('_', expand=True)

kl.head()




In [ ]:
kl_pivot = kl.pivot(index=['record_id', 'visit'], columns = 'side', values='KL-Score').reset_index()

In [ ]:
kl_pivot['diff'] = (kl_pivot['left']-kl_pivot['right']).abs()

In [ ]:
display(kl_pivot[kl_pivot['diff']>1])

Need to remove cases where the kl score for these record_ids and visits is equal to or smaller than 1.0, except for cases like 3022 visit 1.

Might leave these in and out for one or the other and then test which works better. Might also be worth visualizing these questionnaire data.

In [ ]:
sc1 = kl_pivot[(kl_pivot['diff']>1) & (kl_pivot['left']==2.0) & (kl_pivot['right']>kl_pivot['left'])]
display(sc1)

sc2 = kl_pivot[(kl_pivot['diff']>1) & (kl_pivot['right']==2.0) & (kl_pivot['left']>kl_pivot['right'])]
display(sc2)

In [ ]:
kl_pivot = kl_pivot[kl_pivot['diff']>1]

## Check for missing overlap

In [ ]:
def missing_from_df(df1, df2, df1_id, df2_id):
    df1_clean = df1[df1[df1_id].notna()]
    df2_clean = df2[df2[df2_id].notna()]
    
    missing_from_df1 = df2_clean[~df2_clean[df2_id].isin(df1_clean[df1_id])]
    missing_from_df2 = df1_clean[~df1_clean[df1_id].isin(df2_clean[df2_id])]
    
    return missing_from_df1, missing_from_df2


In [ ]:
pi2['name'] = (
    pi2['record_id'].astype(str) + '_' +
    pi2['visit'].astype(str) + '_' +
    pi2['side'].map({'l': 'left', 'r': 'right'})
)


In [ ]:
missing_from_pi2, missing_from_kl = missing_from_df(pi2, kl, 'name', 'name')

print("Ids missing from PI:")
print(missing_from_pi2['name'])
print()
print("Ids missing from KL:")
print(missing_from_kl['name'])

### IDs missing in PI:

* IM2569,IM2590 ,IM2501 ,IM2511: second visit missing
* seem to also be missing UCLA and KOOS scores in some

In [ ]:
id_missingpi = ['IM2569','IM2590' ,'IM2501' ,'IM2511']

pi2[pi2['record_id'].isin(id_missingpi)].sort_values(by= 'record_id')

### IDs missing in KL:

In [ ]:
id_missing = list(set(missing_from_kl['name']))

In [ ]:
print(len(id_missing))
print(len(kl))

In [ ]:
missingklscores = pi2[pi2['name'].isin(id_missing)].sort_values(by='record_id')

if len(missingklscores)>0:
    missingklscores.to_csv(os.path.join(save_dir, "missing_kl_scores.csv"), index = False)

Missing 41 from KL, might be able to use these later, since might be important not to loose to much data.

### Merge Fixed Missing KL Scores with KL, check if still missing first

In [ ]:
df_merged = pi2.merge(kl[['name', 'KL-Score']], on='name', how='left', suffixes=('', '_new'))

df_merged

In [ ]:
missingklscores_edited = pd.read_csv(os.path.join(save_dir2, "missing_kl_scores_edited.csv"))

df_merged = df_merged.merge(missingklscores_edited[['name', 'kl']], on='name', how='left', suffixes=('', '_new'))

df_merged

In [ ]:
display(df_merged[df_merged['kl']>0])

# Replace kl only if a match was found (i.e., kl_new is not NaN)
df_merged['KL-Score'] = df_merged['kl'].combine_first(df_merged['KL-Score'])

# Drop the temporary column

try:
    df_merged.drop(columns= ['kl'], inplace=True)
    df_merged.drop(columns=['kl_new'], inplace=True)
except KeyError:
    print("Column 'kl' or 'kl_new' not found in df_merged. Skipping drop operation.")

In [ ]:
display(df_merged[df_merged['KL-Score'].isna()])

In [ ]:
df_merged = df_merged.dropna(subset='KL-Score')

In [ ]:
df_merged.to_csv(os.path.join(save_dir, 'inmodi_data_personalinformation_kl.csv'), index=False)

## Get Dataframe with Special Cases removed

In [ ]:
kl_pivot['id_visit'] = kl_pivot['record_id'] + '_' + kl_pivot['visit']

In [ ]:
sc1['id_visit'] = sc1['record_id'] + '_' + sc1['visit']
sc2['id_visit'] = sc2['record_id'] + '_' + sc2['visit']

In [ ]:
df_merged
sc1 # keep right leg
sc2 # keep left leg

# Meaning we want to keep the row where side is the same
kl_pivot['side'] = kl_pivot.apply(lambda x: 'l' if x['left']>x['right'] else 'r', axis=1)

# To be removed
kl_pivot['tbr'] = kl_pivot.apply(lambda x: 'l' if x['side']=='r' else 'r', axis = 1)
kl_pivot['visit'] = kl_pivot['visit'].astype(int)

### Remove all, even special cases

In [ ]:
df_filtered = df_merged.merge(kl_pivot, left_on=['record_id', 'visit', 'side'], right_on=['record_id', 'visit', 'tbr'], how='left')
print(f"Number rows before: {len(df_filtered)}")
df_filtered = df_filtered[df_filtered['tbr'].isna()]
print(f"Number rows after: {len(df_filtered)}")


In [ ]:
# To check if any ids were kept.
ids = set(list(kl_pivot['id_visit']))
for id in ids:
    record, visit = id.split('_')

    row = df_filtered[(df_filtered['record_id'] == record) & (df_filtered['visit']== visit)]
    if len(row)>0:
        print(row)

In [ ]:
df_filtered = df_filtered.drop(columns=['left', 'right', 'diff', 'id_visit', 'side_y', 'tbr'])

df_filtered = df_filtered.rename(columns={'side_x':'side'})
display(df_filtered.head())

In [ ]:
df_filtered.to_csv(os.path.join(save_dir, 'inmodi_data_personalinformation_kl_woSC.csv'), index=False)

# MRI Data

In [ ]:
df_filtered.head()

In [ ]:
df_filtered.shape

In [ ]:
mri = pd.read_csv(os.path.join(base_dir, "2025-09-25_mrismall.csv"))

In [ ]:
df_filtered = df_filtered.merge(mri, left_on = 'name', right_on = 'id', suffixes=('', '_mri'), how='left')

In [ ]:
df_filtered.shape

In [ ]:
df_filtered.columns

In [ ]:
df_filtered['KOOS_adl'].max()

In [ ]:
mri_col = ['mri_osteo_yn', 'mri_cart_yn', 'mri_bml_yn']
mri_colname = ['Osteophytes', 'Cartilage', 'Bone Marrow Lesions']

for i, col in enumerate(mri_col):
    sns.set_theme(style="whitegrid", font_scale=1.2)
    plt.figure(figsize=(8, 5))

    # Compute percentages manually
    counts = df_filtered[col].value_counts(normalize=True) * 100

    # Ensure both 0 and 1 appear even if one is missing
    pct = pd.Series([counts.get(0, 0), counts.get(1, 0)], index=[0, 1])

    ax = sns.barplot(
        x=pct.index,
        y=pct.values,
        palette='Set3'
    )

    # Add percentage labels on top
    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{height:.1f}%', 
                    (p.get_x() + p.get_width() / 2., height),
                    ha='center', va='bottom', fontsize=10)

    plt.xlabel(mri_colname[i])
    plt.ylabel("Percentage")
    plt.title(f"Distribution of {mri_colname[i]}")
    plt.ylim(0, 100)
    plt.xticks([0, 1], ["0", "1"])  # force x-axis labels

    plt.show()



# Correlation Analysis

## Heatmap of Correlation

**Pearson Correlation Coefficient**

What it measures:
- The strength and direction of a linear relationship between two continuous variables.

Key Characteristics:
- Sensitive to linear relationships only.
- Values range from -1 to +1.
    - +1: Perfect positive linear relationship
    - 0: No linear relationship
    - -1: Perfect negative linear relationship
- Assumes both variables are normally distributed.
- Affected by outliers.

Use Case:
When you suspect or want to test for a straight-line relationship.

**Spearman Rank Correlation Coefficient**

What it measures:
The strength and direction of a monotonic relationship (doesn't have to be linear) by comparing ranks of the data.

Key Characteristics:
- Non-parametric (does not assume normality).
- Converts raw data to ranks, then applies Pearson's formula to the ranks.
- Captures any monotonic relationship (e.g., curved but consistently increasing or decreasing).
- Less sensitive to outliers than Pearson.

Use Case:
- When the relationship is non-linear but consistently increasing or decreasing.
- Data contains outliers or isn't normally distributed.

**Kendall Tau Correlation Coefficient**

What it measures
The strength and direction of a monotonic relationship based on the number of concordant and discordant pairs.

Key Characteristics:
- Also non-parametric.
- Compares all possible pairs of observations:
- Concordant pair: Both values increase or decrease together.
- Discordant pair: One increases while the other decreases.
- More robust to small sample sizes than Spearman.
- Slightly more conservative (produces smaller absolute values) than Spearman.

Use Case:
Small datasets with ordinal or continuous data.
You want a measure based on the ordering of pairs rather than ranks.

In [ ]:
os.makedirs(os.path.join(save_dir, 'images'), exist_ok=True)

img_save_dir = os.path.join(save_dir, 'images')

### Personal Information

In [ ]:
columns_corr = ['pain', 'age',
       'ce_bmi', 'ce_fm',  'OKS_score', 'UCLA_score',
       'FJS_score', 'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport', 'KOOS_adl',
       'KOOS_qol'] #'COMI_score',
corr_types = ['pearson', 'kendall', 'spearman']

for corr in corr_types:
    print(f"Calculating {corr} correlation...")

    pi2_corr = pi2[columns_corr].corr(method=corr)
    plt.figure(figsize=(12, 8))
    sns.heatmap(pi2_corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
    plt.title(f"{corr.capitalize()} Correlation Heatmap")
    plt.savefig(os.path.join(img_save_dir, f"{corr}corr.png"))
    plt.show()

### With KL-Score

In [ ]:
columns_corr = ['pain', 'age',
       'ce_bmi', 'ce_fm',  'OKS_score', 'UCLA_score',
       'FJS_score', 'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport', 'KOOS_adl',
       'KOOS_qol', 'KL-Score'] #'COMI_score',
corr_types = ['pearson', 'kendall', 'spearman']

for corr in corr_types:
    print(f"Calculating {corr} correlation...")

    comb_corr = df_merged[columns_corr].corr(method=corr)
    plt.figure(figsize=(12, 8))
    sns.heatmap(comb_corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
    plt.title(f"{corr.capitalize()} Correlation Heatmap")
    plt.savefig(os.path.join(img_save_dir, f"{corr}corr_wKL.png"))
    plt.show()

## Pairplot

### Personal Information w/ respect to gender

In [ ]:
col_cat = ['gender']
col_num = ['pain', 'age',
       'ce_bmi', 'ce_fm']
cols = col_cat + col_num

sns.pairplot(pi2[cols], hue = 'gender')
plt.savefig(os.path.join(img_save_dir, "pairplot_personalinformation.png"), bbox_inches='tight')
plt.show()

### Scores w/ respect to gender

In [ ]:
score_values = ['OKS_score', 'UCLA_score',
       'FJS_score', 'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport', 'KOOS_adl',
       'KOOS_qol', 'KL-Score'] #'COMI_score', 

for value in score_values:
    scatterplot(df_merged, x_list=col_num, y=value, hue='gender', figsize = (6, 6), savepath=img_save_dir)

### Personal Information w/respect to pain

In [ ]:
df_merged['pain'].unique()

In [ ]:
pi3 = df_merged.copy()
pi3.dropna(subset=['pain'], axis=0, inplace=True)

pi3['pain']= pi3['pain'].astype(int).astype('str')

In [ ]:
col_num = ['age',
       'ce_bmi', 'ce_fm', 'KL-Score']
cols = col_num

sns.pairplot(pi3, vars=cols, hue = 'pain', hue_order=pi3['pain'].unique(), plot_kws={'alpha': 0.5, 's':30})
plt.savefig(os.path.join(img_save_dir, "pairplot_personalinformation2.png"), bbox_inches='tight')
plt.show()

In [ ]:
sns.boxplot(x='KL-Score', y='pain', data=df_merged)

### Scores w/ respect to pain

In [ ]:
for value in score_values:
    scatterplot(pi3, x_list=col_num, y=value, hue='pain', figsize = (6, 6), savepath=os.path.join(save_dir, 'images'))

### Scores w/ respect to gender

In [ ]:
for value in score_values:
    scatterplot(pi3, x_list=col_num, y=value, hue='gender', figsize = (6, 6), savepath=os.path.join(save_dir, 'images'))

### Scores w/ respect to KL-Score

In [ ]:
col_num = ['age',
       'ce_bmi', 'ce_fm']
for value in score_values:
    scatterplot(pi3, x_list=col_num, y=value, hue='KL-Score', figsize = (6, 6), savepath=os.path.join(save_dir, 'images'))